# Special Methods: Callable

Any object can be emulate a callable implementing `__call__` method. Also usefull when using classes to create decorators

In [2]:
class Person:
    def __call__(self):
        print('__call__ called')

p = Person()

In [3]:
p()

__call__ called


In [4]:
type(p)

__main__.Person

### Partial Callable Class

In [5]:
from functools import partial

In [17]:
def my_func(a, b, c):
    return a, b, c

In [10]:
type(partial), type(Person), type(my_func)

(type, type, function)

In [13]:
partial_func = partial(my_func, 10, 20)
type(partial_func)

functools.partial

In [14]:
partial_func(30)

(10, 20, 30)

In [20]:
class Partial:
    def __init__(self, func, *args):
        self._func = func 
        self._args = args
    
    def __call__(self, *args):
        return self._func(*self._args, *args)

partial_func = Partial(my_func, 10, 20)

In [21]:
type(partial_func)

__main__.Partial

In [22]:
partial_func(30)

(10, 20, 30)

In [24]:
callable(print), callable(int)

(True, True)

In [ ]:
type(print), type(int) #int is also a callable class 

(builtin_function_or_method, type)

In [28]:
type(partial_func), callable(partial_func)

(__main__.Partial, True)

In [30]:
class Person:
    pass 

p = Person()

callable(Person), callable(p)

(True, False)

### Caching mecanism

In [31]:
from collections import defaultdict

In [34]:
def default_value():
    return 'N/A'

d = defaultdict(default_value)

d['a']    

'N/A'

In [35]:
d['b'] = 100

d.items()

dict_items([('a', 'N/A'), ('b', 100)])

In [36]:
miss_counter = 0

def default_value():
    global miss_counter
    miss_counter += 1 
    return 'N/A'

d = defaultdict(default_value)

In [ ]:
d['a'] = 1 
d['b']

d.items(), miss_counter

(dict_items([('a', 1), ('b', 'N/A')]), 1)

In [40]:
def default_value(counter):
    counter += 1  
    return 'N/A'

d = defaultdict(default_value)

d['a']

TypeError: default_value() missing 1 required positional argument: 'counter'

In [41]:
class DefaultValue:
    def __init__(self) -> None:
        self.counter = 0 

    def __iadd__(self, other):
        if isinstance(other, int):
            self.counter += other 
            return self 
        raise ValueError('You can only increment integer values')

In [43]:
default_value_1 = DefaultValue()
default_value_1.counter

0

In [46]:
default_value_1 += 1
default_value_1.counter

3

In [47]:
class DefaultValue:
    def __init__(self) -> None:
        self.counter = 0 

    def __iadd__(self, other):
        if isinstance(other, int):
            self.counter += other 
            return self 
        raise ValueError('You can only increment integer values')
    
    def __call__(self):
        self.counter += 1
        return 'N/A'

In [ ]:
def1 = DefaultValue()
def2 = DefaultValue()

cache1 = defaultdict(def1)
cache2 = defaultdict(def2)

In [50]:
cache1['a'], cache1['b']

('N/A', 'N/A')

In [51]:
def1.counter

2

In [52]:
cache2['a'], cache2['b']

('N/A', 'N/A')

In [53]:
def2.counter

2

In [54]:
class DefaultValue:
    def __init__(self, default_value='N/A') -> None:
        self.counter = 0 
        self.default_value = default_value
        
    def __call__(self):
        self.counter += 1
        return self.default_value

In [56]:
cache_def1 = DefaultValue(None)
cache_def2 = DefaultValue(0)
cache_1 = defaultdict(cache_def1)
cache_2 = defaultdict(cache_def2)

In [57]:
cache_1['a'], cache_1['b']

(None, None)

In [59]:
cache_def1.counter

2

In [60]:
cache_2['a']

0

In [61]:
cache_def2.counter

1

## Creating decorator classes

Problem: How to keep track how long a function has been called and for how long in average?

In [62]:
from time import perf_counter
from functools import wraps

### Raw decorator method

In [73]:
def porfiler(fn):
    _counter = 0 
    _total_elapsed = 0 
    _avg_time = 0 

    @wraps(fn)
    def inner(*args, **kwargs):
        nonlocal _counter 
        nonlocal _total_elapsed
        nonlocal _avg_time

        _counter += 1 
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        _total_elapsed += (end - start)
        _avg_time = _total_elapsed / _counter 
        return result
    
    def counter():
        return _counter
    
    def avg_time():
        return _avg_time
    
    def total_elapsed():
        return _total_elapsed

    inner.counter = counter 
    inner.avg_time = avg_time

    return inner 


In [74]:
from time import sleep
import random

random.seed(0)

@porfiler
def func1():
    sleep(random.random())

In [78]:
func1(), func1()

(None, None)

In [79]:
func1.counter()

4

### Class approach

In [80]:
class Profiler:
    def __init__(self, fn):
        self.counter = 0 
        self.total_elapsed = 0 
        self.fn = fn 

    def __call__(self, *args, **kwargs):
        self.counter += 1 
        start = perf_counter()
        result = self.fn(*args, **kwargs)
        end = perf_counter()
        self.total_elapsed += (end - start)
        return result

    @property
    def avg_time(self):
        return self.total_elapsed / self.counter

In [ ]:
def func1(a, b):
    sleep(random.random())
    return (a + b)

func1 = Profiler(func1)

In [82]:
@Profiler
def func1(a, b):
    sleep(random.random())
    return (a + b)

func1(1, 1)

2

In [86]:
func1.counter, func1.avg_time

(1, 0.5174942999146879)

In [87]:
type(func1)

__main__.Profiler

In [88]:
func1(1, 2)

3

In [89]:
func1.counter

2

In [91]:
func1.avg_time, func1.total_elapsed

(0.4646222499432042, 0.9292444998864084)